In [3]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
import tkinter as tk
from tkinter import scrolledtext

In [4]:
# Load resources
lemmatizer = WordNetLemmatizer()
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ayank\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ayank\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Load trained model and data
model = load_model("models/emotion_model.h5")
intents = json.load(open("Data/responses.json"))
words = pickle.load(open("models/words.pkl", "rb"))
classes = pickle.load(open("models/classes.pkl", "rb"))

In [6]:
# Preprocessing function
def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [7]:
# Convert sentence to bag of words
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

In [8]:
# Predict user emotion
def predict_emotion(sentence):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.05
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [9]:
def get_response(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])

In [10]:
def chatbot_response(msg):
    ints = predict_emotion(msg, model)
    return get_response(ints, intents)

In [11]:
# Get chatbot response
def chatbot_response(text):
    emotions = predict_emotion(text)
    intent = random.choice(emotions)  # Choose the highest-confidence emotion
    for i in intents["intents"]:
        if i["tag"] == intent:
            return random.choice(i["responses"])
    return "I'm here to listen. How are you feeling?"

In [12]:
class ChatBotUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Mental Health Chatbot")
        self.root.geometry("500x600")
        self.root.config(bg="#D1C4E9")  # Soft purple background

        # Chat log with modern styling
        self.chat_log = scrolledtext.ScrolledText(
            root, wrap=tk.WORD, width=55, height=20, state=tk.DISABLED,
            bg="white", fg="black", font=("Arial", 12), padx=10, pady=10
        )
        self.chat_log.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        # User input box
        self.entry_box = tk.Entry(
            root, width=50, bg="white", fg="black", font=("Arial", 12), bd=2, relief=tk.GROOVE
        )
        self.entry_box.pack(pady=10, padx=10, fill=tk.X)
        self.entry_box.bind("<Return>", self.send)

        # Button frame
        self.button_frame = tk.Frame(root, bg="#D1C4E9")
        self.button_frame.pack(pady=5)

        # Send button with modern styling
        self.send_button = tk.Button(
            self.button_frame, text="Send", command=self.send,
            bg="#673AB7", fg="white", font=("Arial", 12, "bold"), bd=2, relief=tk.RAISED
        )
        self.send_button.pack(side=tk.LEFT, padx=5)

        # Theme toggle button
        self.dark_mode = False
        self.theme_button = tk.Button(
            self.button_frame, text="Toggle Theme", command=self.toggle_theme,
            bg="#673AB7", fg="white", font=("Arial", 10, "bold"), bd=2, relief=tk.RAISED
        )
        self.theme_button.pack(side=tk.LEFT, padx=5)

    def send(self, event=None):
        user_msg = self.entry_box.get().strip()
        self.entry_box.delete(0, tk.END)

        if user_msg:
            self.chat_log.config(state=tk.NORMAL)
            self.chat_log.insert(tk.END, f"You: {user_msg}\n", "user")
            bot_response = chatbot_response(user_msg)
            self.chat_log.insert(tk.END, f"Bot: {bot_response}\n\n", "bot")
            self.chat_log.config(state=tk.DISABLED)
            self.chat_log.yview(tk.END)

    def toggle_theme(self):
        """Toggles between light and dark mode."""
        self.dark_mode = not self.dark_mode
        if self.dark_mode:
            self.root.config(bg="#2C2F33")
            self.chat_log.config(bg="#23272A", fg="white", insertbackground="white")
            self.entry_box.config(bg="#99AAB5", fg="black", insertbackground="black")
            self.send_button.config(bg="#7289DA", fg="white")
            self.theme_button.config(bg="#7289DA", fg="white")
        else:
            self.root.config(bg="#D1C4E9")
            self.chat_log.config(bg="white", fg="black", insertbackground="black")
            self.entry_box.config(bg="white", fg="black", insertbackground="black")
            self.send_button.config(bg="#673AB7", fg="white")
            self.theme_button.config(bg="#673AB7", fg="white")


In [13]:
# Run chatbot UI
if __name__ == "__main__":
    root = tk.Tk()
    ChatBotUI(root)
    root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
